In [1]:
%pip install tensorflow
%pip install matplotlib
%pip install pandas
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 8.3/8.3 MB 5.6 MB/s eta 0:00:00
     -------------------------------------- 298.0/298.0 kB 6.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Loading the data

In [1]:
import glob

data_dir = 'data'
data_files = glob.glob(data_dir + "/*.csv")

In [2]:
import pandas as pd

df_list = []

for file in data_files:
    data_frame = pd.read_csv(file, index_col=None, header=0)
    df_list.append(data_frame)

In [3]:
df = pd.concat(df_list, ignore_index=True)

In [4]:
df.shape

(962, 2)

In [5]:
df.columns

Index(['pregunta', 'clase'], dtype='object')

In [6]:
NUMBER_OF_CLASSES = df['clase'].nunique()

In [7]:
df = df.groupby('clase').apply(lambda x: x.sample(n=NUMBER_OF_CLASSES)).reset_index(drop=True)

In [8]:
df.shape

(324, 2)

In [9]:
classes_list = df['clase'].unique().tolist()
print(classes_list)

['Definicion', 'EntdidadGrupo', 'EntidadAbreviatura', 'EntidadLugar', 'EntidadLugarCapital', 'EntidadLugarCiudad', 'EntidadLugarPais', 'EntidadObjeto', 'EntidadPersona', 'Evento', 'NumericoCantidad', 'NumericoEdad', 'NumericoMedida', 'NumericoPeriodo', 'NumericoPorcentaje', 'TemporalAnyo', 'TemporalFecha', 'TemporalMes']


In [10]:
classes_df = pd.DataFrame(columns=classes_list).reset_index(drop=True)
print(classes_df)
print(classes_df.shape)

Empty DataFrame
Columns: [Definicion, EntdidadGrupo, EntidadAbreviatura, EntidadLugar, EntidadLugarCapital, EntidadLugarCiudad, EntidadLugarPais, EntidadObjeto, EntidadPersona, Evento, NumericoCantidad, NumericoEdad, NumericoMedida, NumericoPeriodo, NumericoPorcentaje, TemporalAnyo, TemporalFecha, TemporalMes]
Index: []
(0, 18)


In [11]:
for index, row in df.iterrows():
    classes_df.loc[len(classes_df)] = [1 if row['clase'] == c else 0 for c in classes_list]

In [12]:
classes_df.sample(7)

,Definicion,EntdidadGrupo,EntidadAbreviatura,EntidadLugar,EntidadLugarCapital,EntidadLugarCiudad,EntidadLugarPais,EntidadObjeto,EntidadPersona,Evento,NumericoCantidad,NumericoEdad,NumericoMedida,NumericoPeriodo,NumericoPorcentaje,TemporalAnyo,TemporalFecha,TemporalMes
286,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
80,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
273,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
124,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
188,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
136,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
73,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
question_class_df = pd.concat([df, classes_df], axis=1).drop('clase', axis=1)

In [14]:
question_class_df.sample(7)

,pregunta,Definicion,EntdidadGrupo,EntidadAbreviatura,EntidadLugar,EntidadLugarCapital,EntidadLugarCiudad,EntidadLugarPais,EntidadObjeto,EntidadPersona,Evento,NumericoCantidad,NumericoEdad,NumericoMedida,NumericoPeriodo,NumericoPorcentaje,TemporalAnyo,TemporalFecha,TemporalMes
98,¿En qué ciudad se encuentra la estatua de la L...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
126,¿Cuál es el impacto ambiental de esta industria?,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
21,¿Cuál es el grupo que ha ganado más campeonato...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
307,¿En qué mes se celebra el Día de San Jorge en ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
35,¿Cuál es el grupo que ha ganado más campeonato...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
137,¿Cuál es el objetivo principal de este program...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
170,¿Qué sucedió en la Guerra Fría?,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


# Preprocessing the data

In [15]:
question_class_df['pregunta'] = question_class_df['pregunta'].str.lower()

# Model creation

In [20]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, optimizers
import tensorflow as tf

In [17]:
SEED = 30072001
X_train, x_test, Y_train, y_test = train_test_split(question_class_df['pregunta'], 
                                                    question_class_df[question_class_df.columns[1:]], 
                                                    test_size=0.2, 
                                                    random_state=SEED)

In [18]:
MAX_FEATURES = 2000
SEQUENCE_LENGTH = 150

vectorize_layer = layers.TextVectorization(
    standardize=None, 
    max_tokens=MAX_FEATURES, 
    output_mode='int', 
    output_sequence_length=SEQUENCE_LENGTH)

vectorize_layer.adapt(X_train)

In [99]:
EMBEDDING_DIM = 16

model = tf.keras.Sequential()

model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)
model.add(layers.Embedding(MAX_FEATURES + 1, EMBEDDING_DIM))
model.add(layers.Dropout(0.2))
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(NUMBER_OF_CLASSES, activation="sigmoid"))

model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 150)              0         
 torization)                                                     
                                                                 
 embedding_14 (Embedding)    (None, 150, 16)           32016     
                                                                 
 dropout_13 (Dropout)        (None, 150, 16)           0         
                                                                 
 global_average_pooling1d_11  (None, 16)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_14 (Dropout)        (None, 16)                0         
                                                                 
 dense_19 (Dense)            (None, 18)              

In [100]:
model.compile(optimizer=optimizers.Adam(0.015), loss='binary_crossentropy', metrics=['accuracy' , tf.keras.metrics.AUC()])

In [105]:
history = model.fit(X_train, Y_train,
                    epochs=200,
                    batch_size=64,
                    validation_split=0.05)

Epoch 1/200
4/4 [==============================] - 0s 24ms/step - loss: 0.1580 - accuracy: 0.6098 - auc_14: 0.9078 - val_loss: 0.1748 - val_accuracy: 0.6923 - val_auc_14: 0.8651
Epoch 2/200
4/4 [==============================] - 0s 14ms/step - loss: 0.1591 - accuracy: 0.5976 - auc_14: 0.8927 - val_loss: 0.1735 - val_accuracy: 0.6923 - val_auc_14: 0.8723
Epoch 3/200
4/4 [==============================] - 0s 14ms/step - loss: 0.1549 - accuracy: 0.6545 - auc_14: 0.9123 - val_loss: 0.1728 - val_accuracy: 0.6923 - val_auc_14: 0.8750
Epoch 4/200
4/4 [==============================] - 0s 16ms/step - loss: 0.1562 - accuracy: 0.5772 - auc_14: 0.9062 - val_loss: 0.1721 - val_accuracy: 0.6923 - val_auc_14: 0.8792
Epoch 5/200
4/4 [==============================] - 0s 16ms/step - loss: 0.1512 - accuracy: 0.6382 - auc_14: 0.9159 - val_loss: 0.1713 - val_accuracy: 0.6923 - val_auc_14: 0.8801
Epoch 6/200
4/4 [==============================] - 0s 16ms/step - loss: 0.1509 - accuracy: 0.6626 - auc_14: 0.

In [107]:
metrics = model.evaluate(x_test, y_test)
print("{}: {}".format(model.metrics_names[1], metrics[1]))

3/3 [==============================] - 0s 4ms/step - loss: 0.0749 - accuracy: 0.9538 - auc_14: 0.9878
accuracy: 0.9538461565971375


In [108]:
model.predict(['¿en que año se fundo argentina?'])

1/1 [==============================] - 0s 41ms/step


array([[0.00433052, 0.000981  , 0.00405958, 0.00340125, 0.00262883,
        0.00211475, 0.01089241, 0.00031441, 0.00188384, 0.0023394 ,
        0.00047515, 0.00148069, 0.00356197, 0.00327992, 0.00219901,
        0.09298069, 0.00409826, 0.00225076]], dtype=float32)

In [112]:
model.save('mlp_1.tf', save_format='tf')

INFO:tensorflow:Assets written to: mlp_1.tf\assets


INFO:tensorflow:Assets written to: mlp_1.tf\assets
